##SBERT finetuning
- Stackexchange duplicate dataset https://huggingface.co/datasets/sentence-transformers/stackexchange-duplicates

#### Dependencies

In [ ]:
! pip install \
datasets==3.0.1 \
huggingface-hub==0.25.2 \
sentence-transformers==3.2.0 \
transformers==4.45.2 \
faiss-cpu==1.7.4 \
loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB

#### Import data

In [ ]:
from datasets import load_dataset
sbert_dataset = load_dataset("sentence-transformers/stackexchange-duplicates","post-post-pair")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/250519 [00:00<?, ? examples/s]

In [ ]:
sbert_dataset = sbert_dataset.rename_columns({"post1": "anchor", "post2": "positive"})

In [ ]:
#reduce dataset size
sbert_dataset['train'] = sbert_dataset['train'].select(range(4000))

In [ ]:
sbert_dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4000
    })
})

Create Corpus

In [ ]:
corpus = list(set(sbert_dataset['train']['anchor'] + sbert_dataset['train']['positive']))

In [ ]:
len(corpus)

7455

#### Models to test
- NovaSearch/stella_en_400M_v5
- jinaai/jina-embeddings-v3
- Alibaba-NLP/gte-large-en-v1.5 (TOO LARGE, NOT ENOUGH GPU MEMORY)
- Alibaba-NLP/gte-en-v1.5

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from loguru import logger
import torch
from datasets import Dataset

# Load a pre-trained sentence transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5",
                            trust_remote_code=True,
                            device=device
                            )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cuda


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.3k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

#### Generate negatives

In [ ]:
def generate_corpus_embeddings(model, corpus):

  #Generate embeddings
  logger.info("Generating corpus embeddings...")
  corpus_embeddings = model.encode(corpus,
                                   convert_to_numpy=True,
                                   batch_size=32,  # Adjust batch size based on GPU memory
                                   device=device  # Run on GPU
                                  )

  # Normalize embeddings for cosine similarity
  corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

  #Generate FAISS index
  logger.info("Generating FAISS index...")
  index = faiss.IndexFlatIP(corpus_embeddings.shape[1])  # Inner Product index (cosine similarity after normalization)
  index.add(corpus_embeddings)

  return index

def get_negatives_from_corpus(model,
                              corpus,
                              dataset,
                              index,
                              n_negatives,
                              top_k=20,
                              anchor_col="anchor",
                              positive_col="positive"):

  anchors = dataset[anchor_col]
  positives = dataset[positive_col]

  #Generating query embeddings
  logger.info("Generating query embeddings...")
  query_embeddings = model.encode(anchors,
                                  convert_to_numpy=True,
                                  batch_size=32,  # Adjust batch size based on GPU memory
                                  device=device  # Run on GPU
                                  )

  #Normalizing query embeddings
  query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)

  #Get most similar entries in corpus
  logger.info("Searching FAISS index...")
  similarities, indices = index.search(query_embeddings, top_k)

  #Formatting dataset correctly
  datasets_with_negatives = []
  for i, (anchor, positive) in enumerate(zip(anchors, positives)):

    entry = {anchor_col : anchor,
             positive_col : positive}

    negative_count, j = 0, 0
    while negative_count < n_negatives and negative_count in range(top_k):
      if corpus[indices[i][j]] != anchor and corpus[indices[i][j]] != positive:
        negative_count += 1
        if n_negatives <= 1:
          entry[f"negative"] = corpus[indices[i][j]]
        else:
          entry[f"negative_{negative_count}"] = corpus[indices[i][j]]
      j += 1

    datasets_with_negatives.append(entry)

  logger.info("Dataset with negatives generated")
  return Dataset.from_list(datasets_with_negatives)

In [ ]:
index = generate_corpus_embeddings(model, corpus)

2025-02-05 00:45:46.562 | INFO     | __main__:generate_corpus_embeddings:4 - Generating corpus embeddings...
2025-02-05 00:48:57.062 | INFO     | __main__:generate_corpus_embeddings:15 - Generating FAISS index...


In [ ]:
#If generating embeddings for the entire dataset takes too long, set the dataset to only the eval dataset (valid or test)
queries = sbert_dataset['train']
dataset_with_negatives = get_negatives_from_corpus(model=model,
                                                   corpus=corpus,
                                                   dataset=queries,
                                                   index=index,
                                                   n_negatives=2,
                                                   top_k=20,
                                                   anchor_col="anchor",
                                                   positive_col="positive")

2025-02-05 00:48:57.119 | INFO     | __main__:get_negatives_from_corpus:34 - Generating query embeddings...
2025-02-05 00:50:46.394 | INFO     | __main__:get_negatives_from_corpus:45 - Searching FAISS index...
2025-02-05 00:50:47.670 | INFO     | __main__:get_negatives_from_corpus:67 - Dataset with negatives generated


In [ ]:
dataset_with_negatives

Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2'],
    num_rows: 4000
})

In [ ]:
dataset_with_negatives.to_pandas().head(10)

,anchor,positive,negative_1,negative_2
0,Power Series Solution Consider the initial val...,Convergence of the Power Series for $xu''+\sin...,Find a closed form for the power series $f_p(x...,Prove that the iteration of $\sin(x)$ goes to ...
1,Question on the existence of finite open subse...,Is every open subset of $ \mathbb{R} $ uncount...,Countable closed sets There is a theorem that ...,Is the collection of finite subsets of $\mathb...
2,Hyperdrive vs Warp drive. Which is fastest? In...,Is a warp drive faster than a hyperdrive? What...,"In superluminal phase velocities, what is it t...",How can I maximise the amount of scrap I colle...
3,sed to redirect data to a file I am trying to ...,redirect to the same file as the source file p...,How to ensure that string interpolated into `s...,How can I achieve portability with sed -i (in-...
4,Trying to install Ubuntu 13.04 alongside Windo...,How can I repair grub? (How to get Ubuntu back...,Can't access to Ubuntu 13.04 after installing ...,Unable to install windows 7 over ubuntu 14.04....
5,Removing malware injecting into Google Chrome ...,"How can I remove malicious spyware, malware, a...",How to get rid of this peculiarly nasty pop up...,How to remove Offers4U Ads from Chrome on Mac ...
6,Completely brand new to GIS - Help picking tec...,How to Start Web Mapping? I want to start work...,Working with LiDAR data using other than Esri ...,Generic term for spatial data I'm building an ...
7,Labelling the x-axis \begin{tikzpicture} \...,How to prevent pgfplots from using the 10^n no...,"Plotting data seems ""noisy"" I am trying to plo...",Bar chart - Single colored bar For the bar cha...
8,Asymptotically unbiased estimator vs consisten...,What's the difference between asymptotic unbia...,"Equality in an integral estimate Let $u,v\in W...",What is Degrees of Freedom? I have gone throug...
9,Proving summation identities How would one go ...,General identity for a double summation theore...,"Let $n \in \mathbb{Z}^+$, prove the identity $...",Identity for convolution of central binomial c...


In [ ]:
dataset_with_negatives.to_pandas().to_csv("stackexchange_duplicates_with_negatives.csv", index=False)

# Restart colab here

#### Split dataset

In [ ]:
from datasets import DatasetDict
from datasets import load_dataset

dataset_with_negatives = load_dataset("csv", data_files="stackexchange_duplicates_with_negatives.csv")
train_test_split = dataset_with_negatives['train'].train_test_split(test_size=0.3, seed=42)
test_valid_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

#Restructured sbert_dataset
sbert_dataset = DatasetDict(
    {
      "train" : train_test_split['train'],
      "test" : test_valid_split['train'],
      "validate" : test_valid_split['test']
    }
)

In [ ]:
sbert_dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative_1', 'negative_2'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative_1', 'negative_2'],
        num_rows: 600
    })
    validate: Dataset({
        features: ['anchor', 'positive', 'negative_1', 'negative_2'],
        num_rows: 600
    })
})

#### Training pipeline - wandb api key: 3d9e594db6fdf8e0d50ed596e30b7d54f4a13054

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import MultipleNegativesRankingLoss, CachedMultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from loguru import logger
import torch

# Load a pre-trained sentence transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", #"Alibaba-NLP/gte-large-en-v1.5"
#                             trust_remote_code=True,
#                             device=device
#                             )
savepath="models/gte-en-v1.5-stackexchange-2000/checkpoint-560"
model = SentenceTransformer(savepath,
                            trust_remote_code=True,
                            device=device)

#Define loss function
batch_size = 5
#loss = MultipleNegativesRankingLoss(model) ## Too memory inefficient
loss = CachedMultipleNegativesRankingLoss(model, mini_batch_size=batch_size)

#Training parameters
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/gte-en-v1.5-stackexchange-2000",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    run_name="gte-en-v1.5-stackexchange-2000",  # Will be used in W&B if `wandb` is installed
)

#Create evaluator and evaluate the base model
logger.info("Evaluating base model on validation dataset...")
dev_evaluator = TripletEvaluator(
    anchors=sbert_dataset["validate"]["anchor"],
    positives=sbert_dataset["validate"]["positive"],
    negatives=sbert_dataset["validate"]["negative_1"],
    name="base-eval-val",
)
base_results = dev_evaluator(model)
logger.info(f"Base model results: {base_results}")

#Create trainer and train
logger.info("Training model...")
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=sbert_dataset["train"].select_columns(["anchor", "positive"]),
    eval_dataset=sbert_dataset["validate"],
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

# Save model
logger.info("Saving model...")
model.save_pretrained("models/gte-en-v1.5-stackexchange-2000/final")

2025-02-05 01:36:58.476 | INFO     | __main__:<cell line: 0>:16 - Using device: cuda
2025-02-05 01:36:59.691 | INFO     | __main__:<cell line: 0>:56 - Evaluating base model on validation dataset...
2025-02-05 01:37:46.334 | INFO     | __main__:<cell line: 0>:64 - Base model results: {'base-eval-val_cosine_accuracy': 0.6833333333333333, 'base-eval-val_dot_accuracy': 0.3233333333333333, 'base-eval-val_manhattan_accuracy': 0.6783333333333333, 'base-eval-val_euclidean_accuracy': 0.68, 'base-eval-val_max_accuracy': 0.6833333333333333}
2025-02-05 01:37:46.336 | INFO     | __main__:<cell line: 0>:67 - Training model...
wandb: Currently logged in as: thefunnyonion (thefunnyonion-limindustries) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Base-eval-val Cosine Accuracy,Base-eval-val Dot Accuracy,Base-eval-val Manhattan Accuracy,Base-eval-val Euclidean Accuracy,Base-eval-val Max Accuracy
10,0.006200,1.050973,0.681667,0.321667,0.678333,0.678333,0.681667
20,0.000300,1.052991,0.678333,0.321667,0.678333,0.678333,0.678333
30,0.011400,1.054278,0.676667,0.318333,0.675000,0.678333,0.678333
40,0.000200,1.054078,0.676667,0.318333,0.675000,0.680000,0.680000
50,0.000100,1.051784,0.681667,0.318333,0.673333,0.681667,0.681667
60,0.000200,1.036165,0.680000,0.318333,0.675000,0.681667,0.681667
70,0.000400,1.029793,0.675000,0.321667,0.673333,0.678333,0.678333
80,0.000300,1.025900,0.673333,0.326667,0.668333,0.675000,0.675000
90,0.000100,1.024392,0.676667,0.323333,0.671667,0.678333,0.678333
100,0.000100,1.023098,0.676667,0.320000,0.671667,0.680000,0.680000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

2025-02-05 02:17:21.745 | INFO     | __main__:<cell line: 0>:79 - Saving model...


In [ ]:
# (Optional) Evaluate the trained model on the test set
test_evaluator = TripletEvaluator(
    anchors=sbert_dataset["test"]["anchor"],
    positives=sbert_dataset["test"]["positive"],
    negatives=sbert_dataset["test"]["negative_1"],
    name="base-eval-test",
)
test_evaluator(model)

{'base-eval-test_cosine_accuracy': 0.6733333333333333,
 'base-eval-test_dot_accuracy': 0.3283333333333333,
 'base-eval-test_manhattan_accuracy': 0.6716666666666666,
 'base-eval-test_euclidean_accuracy': 0.675,
 'base-eval-test_max_accuracy': 0.675}